In [ ]:
!pip install python-dotenv

# Korean

In [ ]:
import os
import re
import tiktoken
from openai import OpenAI
from dotenv import load_dotenv
import ast
import pandas as pd
import json

# Load environment variables
load_dotenv()
client = OpenAI(api_key=(OPENAI_API_KEY))
enc = tiktoken.get_encoding("cl100k_base")

# =========================================================
# 1. Character Extraction (KOREAN)
# =========================================================

def character_extraction_korean(input_text: str):

    prompt = f"""
You are an expert in Korean traditional folktale character tagging.

Given the following input text, extract main characters and generate:
1. JSON object `character_list` in this format:
   {{ "Simcheong" : ["main-girl", "A girl wearing an old korean traditional skirt, with her hair braided to one side."] }}

Rules:

- Only extract characters essential to story progression. There must be only one character, not multiple characters.
- character_name must be a single-syllable Korean-style name with no modifiers.
- reference_name = role-gender_age (ex: main-girl, sub-oldman)
- If the character is a non-human animal, use the English name of the animal as the reference_name (e.g., "Tiger", "Frog").
- If the character is a non-human proper noun (like Dokkaebi or Gumiho), use its romanized Korean name as reference_name (e.g., "Dokkaebi").
- If appearance is not present in the input, infer it using traditional Korean clothing and style.
- Use elements like jeogori, durumagi, binyeo, gat, norigae, etc.

examples : {{
  "Simcheong": ["main-girl", "A girl wearing an old hanbok, her hair braided to one side."],
  "BlindFather": ["main-oldman", "An elderly man in a worn durumagi, holding a wooden cane and wearing a topknot."],
  "LotusSpirit": ["spirit", "A glowing spirit figure with a lotus flower crown, wearing translucent hanbok robes."],
  "Monk": ["main-man", "A bald man in simple robes, carrying prayer beads and a walking stick."],
  "MarketWoman": ["sub-woman", "A middle-aged woman in a jeogori, carrying a woven basket of herbs."],
  "Tiger": ["Tiger", "A large, fierce-looking tiger with expressive eyes, standing on a rocky forest path."],
  "Scholar": ["sub-nobleman", "A young man in scholarly robes and a gat, holding a scroll."],
  "Dokkaebi": ["goblin", "A mischievous goblin with wild fur, carrying a spiked club and wearing tattered pants."],
  "Gisaeng": ["sub-woman", "A graceful woman in a hanbok, holding a fan and wearing elaborate hairpins."],
  "King": ["main-oldman", "A dignified man in a royal robe with embroidery, wearing an ikseongwan hat, seated on a wooden throne with a calm expression."]
}}

2. Replace all characters in the input with reference_name, including demonstrative pronouns.

Return JSON format only.
Input:
\"\"\"{input_text.strip()}\"\"\"
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )

    content = response.choices[0].message.content.strip()

    json_match = re.search(r'\{.*\}', content, re.DOTALL)
    if not json_match:
        raise ValueError("No JSON block found in GPT response.")

    parsed = json.loads(json_match.group(0))
    return parsed["character_list"], parsed["replaced_text"]


# =========================================================
# 2. Split Text
# =========================================================

def split_text_korean(replaced_text: str, num_sections: int = 3):

    section_names = [f"section_{i+1}" for i in range(num_sections)]

    prompt = f"""
You are an expert in narrative segmentation.

Split the following Korean folktale into {num_sections} coherent sections
based on narrative or emotional flow.

Return exactly in this format:
{chr(10).join([f"{name}:" for name in section_names])}

Input:
\"\"\"{replaced_text.strip()}\"\"\"
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )

    content = response.choices[0].message.content.strip()

    sections = {}
    for i, name in enumerate(section_names):
        next_name = section_names[i + 1] if i + 1 < num_sections else None
        pattern = (
            rf'{name}[\s:]*\n*(.*?)\n(?={next_name}|$)'
            if next_name else
            rf'{name}[\s:]*\n*(.*)'
        )
        match = re.search(pattern, content, re.DOTALL | re.IGNORECASE)
        if match:
            sections[name] = match.group(1).strip()

    if not sections:
        split_size = len(replaced_text) // num_sections
        for i in range(num_sections):
            start = i * split_size
            end = (i + 1) * split_size if i + 1 < num_sections else len(replaced_text)
            sections[f"section_{i+1}"] = replaced_text[start:end].strip()

    return sections


# =========================================================
# 3. T2I Prompt
# =========================================================

def generate_t2i_prompt_korean(section_text: str, character_list: dict, section_name: str):

    prompt = f"""
You are a T2I prompt engineer creating a detailed scene description from the [{section_name}] part of a story.

Your goal is to describe a single still visual scene for a text-to-image generation model.
Select a single important moment from the story based on its narrative structure.
This moment should reflect a major emotional shift, conflict, resolution, or symbolic turning point in the narrative.

Instructions:
Your task is to generate a concise, vivid scene description for a text-to-image model.

Rules:
- You must express dynamic. and one scene must have one character
1. Use only reference_names. Do NOT use character names. In each scene, describe only one main character
2. For each important object (character, animal, structure, tree, etc.), describe its appearance, emotion or action, and key attributes in one sentence. Include its spatial location (e.g., Center., Left., Back right., etc.). Format: [reference_name] [detailed description]. [Position.]
3. Keep descriptions vivid but compact (1 sentence each).
4. Describe background elements (e.g., hanok house, stone lantern, pine tree, river) and their locations.
5. Always mention the main character first in the scene.
6. The full prompt should be between 30 to  40 words, and feel like a still frame from a painting.
7. Use simple, clear English with short, direct phrases (e.g., Standing, Smiling, Holding). Avoid complex sentences and abstract expressions.
8. Maintain the order: main character → supporting objects/characters → background → style.
9. Always specify spatial location for each element.
10. Eliminate repeated phrases and unnecessary adjectives.
11. At the end of every prompt, include: kimhongdo style, ink wash painting, muted earthy tones, subtle color highlights, thin line
12. Arrange background elements so they do not overlap spatially with characters.

Here are a few examples of well-written visual prompts:
1 sub-boy in a torn hanbok dashes barefoot, clutching a scroll. Right middle. A cracked fortress wall looms with ivy crawling up. Back center. Kimhongdo style, Joseon Dynasty, ink wash painting.
2 Main-girl wears a jeogori and chima, running in forest. Center. Kimhongdo style, Joseon Dynasty, ink wash painting.
3 main-girl in a torn hanbok kneels at a grave mound, placing wildflowers gently. Center. A full moon glows behind a pine tree. Back left. Kimhongdo style, Joseon Dynasty, ink wash painting.
4 sub-oldman with a fishing rod stands ankle-deep in the stream, lifting a net with a single fish. Right front. His straw hat shades his eyes. Kimhongdo style, Joseon Dynasty, ink wash painting.
5 sub-woman in a jeogori balances a large jar on her head while stepping over stones in a shallow stream. Right. A willow tree droops gently overhead. Back left. Kimhongdo style, Joseon Dynasty, ink wash painting.
6 goblin with wild fur, twisted horns, and a jagged club crouches behind a stone wall, grinning mischievously. Left. A full moon shines above a crumbling village gate. Back center. Kimhongdo style, Joseon Dynasty, ink wash painting.
7 Tiger with bold stripes and calm eyes walks slowly past a tall rock, tail swaying gently. Center. Bamboo leaves rustle in the wind. Back right. Kimhongdo style, Joseon Dynasty, ink wash painting.
8 Fox with soft fur and a bushy tail peeks out from behind a stack of firewood, alert and cautious. Left. A small thatched cottage stands quietly under bare branches. Back center. Kimhongdo style, Joseon Dynasty, ink wash painting.
9 main-woman in a jeogori and skirt sits quietly, sewing under a paper lantern. Center. A wooden chest and folded clothes lie beside her. Back right. Kimhongdo style, Joseon Dynasty, ink wash painting.
10 main-man in worn scholar robes clutches a scroll with both hands, gazing toward distant mountains. Right. A tall pine tree bends in the wind. Back left. Kimhongdo style, Joseon Dynasty, ink wash painting.


Now generate your own visual description using the character list and section text below.

character_list = {json.dumps(character_list, ensure_ascii=False, indent=2)}

section_text = \"\"\"{section_text}\"\"\"
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content.strip()


# =========================================================
# 4. Pipeline
# =========================================================

def run_pipeline_korean(input_text: str, num_sections: int = 3):

    print("Step 1: Extracting Characters...")
    character_list, replaced_text = character_extraction_korean(input_text)

    print("Step 2: Splitting Text...")
    sections = split_text_korean(replaced_text, num_sections)

    print("Step 3: Generating Prompts...")
    image_prompts = {}

    for name, text in sections.items():
        image_prompts[name] = generate_t2i_prompt_korean(text, character_list, name)

    return {
        "character_list": character_list,
        "replaced_text": replaced_text,
        "split_texts": sections,
        "image_prompts": image_prompts
    }

In [ ]:
if __name__ == "__main__":
    input_text = """
옛날 옛적, 산골 마을에 무시무시한 도깨비가 살았단다.
그 도깨비는 키가 크고, 새카만 얼굴에, 이빨은 송곳같이 뾰족했단다.
온몸은 붉은 털로 덮여 있었고, 머리는 말처럼 길쭉했지.
도깨비는 마을 어귀의 커다란 느티나무, 신목 속에 산다고들 하였단다.
지나는 이마다 "신목님, 안녕하십니까" 하고 고개 숙여 인사해야 했지.
그렇지 않으면 도깨비가 밤에 나타나 돌을 던져 혼을 내곤 했단다.

바로 그 마을에 순하고 마음씨 고운 소녀, '순이'가 살았어.
순이는 언제나 느티나무 앞을 지날 때마다 정성껏 인사하였지.
어느 날, 순이는 집에 쓸 장작을 주우러 숲속에 홀로 들어갔단다.
땅에 떨어진 마른 나뭇가지를 한 아름 주워, 단단히 묶어 머리에 이고
노래를 흥얼거리며 집으로 돌아오고 있었어.
그런데 그만, 신목 앞에서 인사를 깜빡 잊었단다.

그때 갑자기, 숲속이 쿵쾅 울리며 무서운 소리가 들려왔지.
도깨비가 바람을 타고 날아와, 순이를 홱 잡아채더니
느티나무 속에 가두어 버렸단다.

이틀이 지나도 순이가 돌아오지 않자, 부모님은 애가 탔단다.
마을 사람들도 모두 모여 산과 들을 샅샅이 찾아 헤맸지.
아무리 불러도 소식이 없었어.
그때 한 마을 사람이 숲속에서 순이의 단정하게 묶인 장작 더미를 발견했단다.
기쁜 소식에 모두 모여들었는데, 그때 갑자기 느티나무 안에서
'쿵, 쿵'하고 두드리는 소리가 들렸지 뭐니?

사람들은 깜짝 놀랐지만, 용감한 이들도 있었단다.
"저 안에서 순이가 우리를 부르는 게야!"
모두 순이를 구하려 하였지.

어른들은 촛불과 꽹과리, 장구를 가져오고,
순이 어머니는 하늘님께 빌며, 작은 나무 십자가 대신 부처님 탑 모양의 부적을 들고 나섰단다.
모두 신목을 둥그렇게 둘러싸고,
촛불을 밝히고 농악대가 신명나게 장단을 울렸지.
순이 어머니가 부적을 들고 신목을 조심스레 두드렸단다.

그 순간, 산이 쩌렁쩌렁 울리며 무서운 굉음이 나고
느티나무 신목이 쩍 갈라졌어!
그 속에서 방긋 웃는 순이가 멀쩡히 나왔단다.

순이는 어머니 품에 안기고,
농악대는 신나게 연주하였지.
마을 사람들은 모두 손뼉 치며 기뻐하였단다.
그날 밤, 온 마을이 모여 흥겹게 노래하고 춤을 추며 밤새도록 기쁨을 나누었지.

지금도 그 느티나무 신목은 마을에 우뚝 서 있지만,
사람들은 더는 두려워하지 않고,
도깨비도 다시는 마을사람을 괴롭히지 못했단다.
"""
    result = run_pipeline_korean(input_text, num_sections=3)



Step 1: Extracting Characters...
Step 2: Splitting Text...
Step 3: Generating Prompts...


In [ ]:
import json
print(json.dumps(result, ensure_ascii=False, indent=2))

{
  "character_list": {
    "Dokkaebi": [
      "goblin",
      "A tall goblin with a jet-black face and sharp teeth, covered in red fur with a horse-like elongated head."
    ],
    "Soony": [
      "main-girl",
      "A gentle-hearted girl in a traditional hanbok with her hair braided."
    ],
    "Mother": [
      "sub-oldwoman",
      "An elderly woman wearing a simple jeogori, holding a Buddha-shaped charm."
    ]
  },
  "replaced_text": "옛날 옛적, 산골 마을에 무시무시한 Dokkaebi가 살았단다. 그 Dokkaebi는 키가 크고, 새카만 얼굴에, 이빨은 송곳같이 뾰족했단다. 온몸은 붉은 털로 덮여 있었고, 머리는 말처럼 길쭉했지. Dokkaebi는 마을 어귀의 커다란 느티나무, 신목 속에 산다고들 하였단다. 지나는 이마다 '신목님, 안녕하십니까' 하고 고개 숙여 인사해야 했지. 그렇지 않으면 Dokkaebi가 밤에 나타나 돌을 던져 혼을 내곤 했단다.\n\n바로 그 마을에 순하고 마음씨 고운 소녀, 'Soony'가 살았어. Soony는 언제나 느티나무 앞을 지날 때마다 정성껏 인사하였지. 어느 날, Soony는 집에 쓸 장작을 주우러 숲속에 홀로 들어갔단다. 땅에 떨어진 마른 나뭇가지를 한 아름 주워, 단단히 묶어 머리에 이고 노래를 흥얼거리며 집으로 돌아오고 있었어. 그런데 그만, 신목 앞에서 인사를 깜빡 잊었단다.\n\n그때 갑자기, 숲속이 쿵쾅 울리며 무서운 소리가 들려왔지. Dokkaebi가 바람을 타고 날아와, Soony를 홱 잡아채더니 느티나무 속에 가두어 버렸단다.\n\n이틀이 지나도 Soo

# Bengal

In [ ]:
import os
import re
import tiktoken
from openai import OpenAI
from dotenv import load_dotenv
import json

# =========================================================
# Environment Setup
# =========================================================

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
enc = tiktoken.get_encoding("cl100k_base")


# =========================================================
# 1. Character Extraction
# =========================================================

def character_extraction(input_text: str):

    prompt = f"""
You are an expert in Bengali traditional folktale character tagging.

Given the following input text, extract main characters and generate:
1. JSON object `character_list` in this format:
   {{ "name" : ["reference_name", "appearance description"] }}

Rules:
[Character Extraction]
- Only extract characters essential to story progression.
- Do NOT treat background elements (trees, huts, rivers, mountains, temples, fields, etc.) as characters.
- reference_name = role-gender_age (ex: main-girl, sub-oldman).
- If the character is a non-human animal, use the English name of the animal as the reference_name.
- If mythological being (Rakshasa, Naga, Hanuman), use that as reference_name.
- The character description must always include a natural and vivid external appearance.
- Do NOT include actions, emotions, personality traits, or background objects.

[description]
- Only include external appearance.
- Use elements like jamdani sari, braid with flowers, alta, shankha, pola, gold nose ring, kurta, dhoti, lungi, gamcha, turban, rudraksha mala.

Return JSON format:

```json
{{
  "character_list": {{
    ...
  }},
  "replaced_text": "..."
}}

Input:
"""{input_text.strip()}"""
"""
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": prompt}]
)

content = response.choices[0].message.content.strip()

try:
    json_match = re.search(r'```json\s*(\{.*?\})\s*```', content, re.DOTALL)
    if not json_match:
        raise ValueError("No JSON block found in GPT response.")

    parsed_json = json.loads(json_match.group(1))

    character_list = parsed_json["character_list"]
    replaced_text = parsed_json["replaced_text"]

    return character_list, replaced_text

except Exception as e:
    print("Parsing failed:", e)
    print("Raw content:\n", content)
    raise ValueError( "Failed to extract character_list or replaced_text.")


In [ ]:
if __name__ == "__main__":
    folktale_text = """
সন্ধ্যায় ফিরল ঘরের মালিকেরা। সাতজন তাঁতী, খাটো গড়ন। ধুতি, গামছা, মুঠোভর্তি সূতা। ঘর দেখে তারা অবাক। কথা কাটাকাটি হল একটু। শেষে বলল, থাকো বোন, থাকো। অপর্ণা বলল, আমি রান্না করব। ঘর দেখব। সে রাঁধল ভাত, ডাল, শাক। পুঁটির ঝোলে সরষের গন্ধ। ঘর ভরে উঠল সুখে।

"""
    result = run_pipeline_bengal(folktale_text, num_sections=1)

import json
print(json.dumps(result, ensure_ascii=False, indent=2))



Step 1: Extracting Characters...
✅ Extracted 2 characters.

Step 2: Splitting Text into 1 sections...

🔍 GPT split_text raw output preview:
 section_1: 

In the evening, the sub-men group returned home. The sub-men group were short-statured, wearing dhotis, gamchas, and carrying bunches of thread. Seeing their home, the sub-men group were astonished. Some arguments ensued. Finally, the sub-men group said, stay, sister, stay. The main girl said, I will cook. I will manage the house. The main girl cooked rice, dal, and greens. The fish curry smelled of mustard. The house filled with happiness. ...

✅ Successfully split into 1 sections.

Step 3: Generating T2I Prompts...
  - Generating prompt for section_1...

✅ Pipeline Complete!
{
  "character_list": {
    "main-girl": [
      "main-girl",
      "A young woman in a simple sari, with her hair tied in a neat braid, wearing minimal jewelry."
    ],
    "weavers": [
      "sub-men_group",
      "A group of seven short-statured men, wearing 